In [2]:
import os
import imageio
import numpy as np

In [3]:
path = '../ColabNotebooks/dataset'

In [4]:
def extractFrames(file_name, frame_step):
  video = imageio.get_reader(file_name, 'ffmpeg')
  return np.array([frame for i, frame in enumerate(video) if i % frame_step == 0])

In [5]:
def preProcessing(frames):
  video_r_channel = np.zeros(frames.shape[0:3])
  video_g_channel = np.zeros(frames.shape[0:3])
  video_b_channel = np.zeros(frames.shape[0:3])

  for i, frame in enumerate(frames):
    for j, row in enumerate(frame):
      for k, pixel in enumerate(row):
        video_r_channel[i][j][k] = pixel[0]
        video_g_channel[i][j][k] = pixel[1]
        video_b_channel[i][j][k] = pixel[2]
        
  return video_r_channel, video_g_channel, video_b_channel

In [6]:
# Returns correlation value
def superimpose(frame: np.ndarray, kernel: np.ndarray):
  n = kernel.shape[0]
  mid = n // 2

  superimposed_matrix = frame * kernel
  return sum(superimposed_matrix.flatten()) - superimposed_matrix[mid][mid]

In [7]:
def isValid(index, length, skip):
  return index >= skip and index < length - skip

In [8]:
# Channel is a 3D matrix: (number of frames, height, width)
# each element represents the pixel value of the given channel

def convolve(channel, kernel):
  n = len(kernel) # Kernel is of dimension n x n
  skip = n // 2 # No. of rows and cols to be skipped

  num_frames, height, width = channel.shape

  # res_frames = np.zeros((len(frames), height - 2 * skip, width - 2 * skip))
  res_channel = np.zeros(channel.shape)

  for i, frame in enumerate(channel):
    print(f'Frame: {i}')

    for j, row in enumerate(frame):
      if not isValid(j, height, skip):
        continue

      for k, pixel in enumerate(row):
        if not isValid(k, width, skip):
          continue

        subframe = frame[j - skip: j + skip + 1, k - skip: k + skip + 1]

        res_channel[i][j][k] = superimpose(subframe, kernel)

  return res_channel[:, skip:-skip, skip:-skip]

In [9]:
def saveAsVideo(frames, file_name, fps):
  frames = frames.astype('uint8')
  writer = imageio.get_writer(file_name, fps=fps, macro_block_size=None)
  for frame in frames:
    writer.append_data(frame)

In [14]:
#driver code
files = os.listdir(path)

frames = np.array(extractFrames(os.path.join(path, files[5]), 30))
video_r_channel, video_g_channel, video_b_channel = preProcessing(frames)

In [ ]:
saveAsVideo(video_r_channel, 'video_r.mp4', 5)
saveAsVideo(video_g_channel, 'video_g.mp4', 5)
saveAsVideo(video_b_channel, 'video_b.mp4', 5)

In [15]:
tmpframe = np.array([[[2,2,2,4,5,6], [2,2,2,4,5,6], [2,2,2,4,5,6], [2,2,2,4,5,6], [1,2,3,4,5,6]]])
print(tmpframe.shape)

# tmpkernel = np.ones((3,3))
tmpkernel = np.full((3,3), 1/9)

tmpa = convolve(tmpframe, tmpkernel)
print(tmpa)

(1, 5, 6)
Frame: 0
[[[1.77777778 2.44444444 3.22222222 4.44444444]
  [1.77777778 2.44444444 3.22222222 4.44444444]
  [1.77777778 2.55555556 3.33333333 4.44444444]]]


In [16]:
kernel = np.full((3,3), 1/9)
actual = video_r_channel[0][1:-1, 1:-1]
actual

array([[163., 163., 163., ..., 158., 158., 158.],
       [163., 163., 163., ..., 158., 158., 158.],
       [163., 163., 163., ..., 159., 159., 159.],
       ...,
       [ 52.,  51.,  51., ..., 253., 253., 253.],
       [ 52.,  52.,  52., ..., 253., 253., 253.],
       [ 52.,  52.,  52., ..., 253., 253., 253.]])

In [ ]:
predicted = convolve(np.array([video_r_channel[0]]), kernel).astype('uint8')
predicted

Frame: 0


array([[[144, 144, 144, ..., 140, 140, 140],
        [144, 144, 144, ..., 140, 140, 140],
        [144, 144, 144, ..., 140, 140, 140],
        ...,
        [ 45,  45,  45, ..., 224, 224, 224],
        [ 46,  46,  45, ..., 224, 224, 224],
        [ 47,  46,  46, ..., 224, 224, 224]]], dtype=uint8)

In [ ]:
imageio.imwrite('predicted_image_r.jpg', predicted[0])

In [ ]:
diff = abs(actual - predicted[0]).astype('uint8')
diff

array([[19, 19, 19, ..., 18, 18, 18],
       [19, 19, 19, ..., 18, 18, 18],
       [19, 19, 19, ..., 19, 19, 19],
       ...,
       [ 7,  6,  6, ..., 29, 29, 29],
       [ 6,  6,  7, ..., 29, 29, 29],
       [ 5,  6,  6, ..., 29, 29, 29]], dtype=uint8)

In [ ]:
(1 / len(diff.flatten())) * sum(diff.flatten())

13.366895691824933

In [ ]:
imageio.imwrite('error_image_r.jpg', diff)

# Below Code is Work-in-Progress

In [95]:
from sklearn.mixture import GaussianMixture
import random

In [49]:
import scipy.stats
tmpObj = scipy.stats.norm(loc=tmpMeans, scale=tmpStddev)
tmpObj.pdf(2.9999)

0.2820947205449703

In [23]:
video_b_channel[0]

array([[128., 128., 128., ..., 125., 125., 125.],
       [128., 128., 128., ..., 125., 125., 125.],
       [128., 128., 128., ..., 125., 125., 125.],
       ...,
       [ 41.,  39.,  39., ..., 246., 246., 246.],
       [ 41.,  39.,  39., ..., 246., 246., 246.],
       [ 42.,  41.,  41., ..., 246., 246., 246.]])

In [81]:
def get_probability(X, y):
  model = GaussianMixture(n_components=1, random_state=45).fit(X)
  mean = model.means_.flatten()[0]
  stddev = (model.covariances_ ** (1/2)).flatten()[0]

  # print(f'mean = {mean}, stddev = {stddev}')
  # print(scipy.stats.norm(loc=mean, scale=stddev).pdf(y))
  return scipy.stats.norm(loc=mean, scale=stddev).pdf(y)

In [90]:
gm = GaussianMixture(n_components=1, random_state=45)
tmpX = np.array([[162.9], [163.8], [163], [163], [163], [163], [163.1], [163]]) + np.random.rand(1)[0]
get_probability(tmpX, 163)

0.02912691621949884

In [99]:
np.array([1, 2, 3]).reshape(-1, 1) + np.random.rand(3).reshape(-1, 1)

array([[1.16857431],
       [2.58533249],
       [3.07026451]])

In [89]:
video_b_channel[0]

array([[128., 128., 128., ..., 125., 125., 125.],
       [128., 128., 128., ..., 125., 125., 125.],
       [128., 128., 128., ..., 125., 125., 125.],
       ...,
       [ 41.,  39.,  39., ..., 246., 246., 246.],
       [ 41.,  39.,  39., ..., 246., 246., 246.],
       [ 42.,  41.,  41., ..., 246., 246., 246.]])

In [112]:
def get_probability_matrix(frame, kernel_size):
  skip = kernel_size // 2 # No. of rows and cols to be skipped

  height, width = frame.shape

  res = np.zeros(frame.shape, dtype=float)

  for i, row in enumerate(frame):
    print(i)
    if not isValid(i, height, skip):
      continue

    for j, pixel in enumerate(row):
      if not isValid(j, width, skip):
        continue

      subframe = frame[i - skip: i + skip + 1, j - skip: j + skip + 1]

      X = np.delete(subframe.flatten(), skip * kernel_size + skip).reshape(-1, 1) + np.random.rand(kernel_size ** 2 - 1).reshape(-1, 1)

      res[i][j] = get_probability(X, pixel)

      # print(res[i][j])

  return res[skip:-skip, skip:-skip]


In [114]:
res = get_probability_matrix(video_b_channel[0][:432, 576:1344], 3)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [115]:
res

array([[0.22285649, 0.37223773, 0.41945588, ..., 0.34838355, 0.7085947 ,
        0.12658127],
       [0.23251776, 0.33359534, 0.32723953, ..., 0.13534093, 0.47995739,
        0.11858373],
       [0.38568904, 0.37404716, 0.47537406, ..., 0.40477637, 0.29228567,
        0.25013454],
       ...,
       [0.63575505, 0.60258685, 0.8876449 , ..., 0.20588727, 0.14560069,
        0.16911012],
       [0.53724847, 0.1200192 , 0.71651454, ..., 0.2128023 , 0.14898118,
        0.18007654],
       [0.38503271, 0.26354151, 0.06286265, ..., 0.223136  , 0.15583313,
        0.16324829]])

In [122]:
err = (res > 0.1).astype('uint8') * 255

In [123]:
imageio.imwrite('error_visualized.jpg', err)

## Original Image

In [129]:
file_ind = files.index('bxzakyopjf.mp4')
file_ind

170

In [130]:
org_frames = np.array(extractFrames(os.path.join(path, files[file_ind]), 30))
org_video_r_channel, org_video_g_channel, org_video_b_channel = preProcessing(org_frames)

In [131]:
org_res = get_probability_matrix(org_video_b_channel[0][155:669, 768:1152], 3)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [132]:
org_err = (org_res > 0.1).astype('uint8') * 255
org_err

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0, 255, ...,   0,   0,   0],
       ...,
       [255, 255, 255, ..., 255,   0,   0],
       [255,   0, 255, ..., 255,   0,   0],
       [255, 255, 255, ..., 255,   0,   0]], dtype=uint8)

In [133]:
imageio.imwrite('org_error_visualized.jpg', org_err)

In [134]:
fake_file_ind = files.index('bourlmzsio.mp4')
fake_file_ind

134

In [135]:
fake_frames = np.array(extractFrames(os.path.join(path, files[fake_file_ind]), 30))
fake_video_r_channel, fake_video_g_channel, fake_video_b_channel = preProcessing(fake_frames)

In [136]:
fake_res = get_probability_matrix(fake_video_b_channel[0][155:669, 768:1152], 3)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [137]:
fake_err = (fake_res > 0.1).astype('uint8') * 255
fake_err

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0, 255, ...,   0,   0,   0],
       [  0,   0, 255, ...,   0,   0,   0],
       ...,
       [255, 255, 255, ..., 255,   0,   0],
       [255, 255, 255, ..., 255,   0,   0],
       [255, 255, 255, ..., 255,   0,   0]], dtype=uint8)

In [139]:
imageio.imwrite('fake_error_visualized.jpg', fake_err)

In [140]:
! git add .

In [141]:
! git commit -m "probability calc"

[main a3183f0] probability calc
 7 files changed, 24 insertions(+), 1 deletion(-)
 create mode 100644 error_image_g.jpg
 create mode 100644 error_visualized.jpg
 create mode 100644 fake_error_visualized.jpg
 create mode 100644 org_error_visualized.jpg
 create mode 100644 predicted_image.jpg
 create mode 100644 predicted_image_g.jpg


In [142]:
! git push

Counting objects: 9, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (9/9), done.
Writing objects: 100% (9/9), 588.36 KiB | 13.68 MiB/s, done.
Total 9 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/Tony-Stark007/DeepFakeDetection.git
   ee63363..a3183f0  main -> main
